In [1]:
!uname -a


Linux e937effdf510 6.1.58+ #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux


In [2]:

!pip install wandb nltk torchinfo matplotlib==3.8.2 torch==2.1.2 torchvision==0.16.2 numpy pandas  python-dotenv psycopg2-binary  pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 28.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 6.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 5.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 14.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00:00:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:0000:0100:02m
     ━━

In [2]:
import torch
import torch.nn as nn  # (i.e. "neural network")
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import wandb
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

import tensorflow_hub as hub
USE = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")



2024-01-31 23:13:40.075846: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 23:13:40.075979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 23:13:40.082164: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 23:13:42.880947: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from torch.utils.data import DataLoader


In [5]:

# FILENAME = "./downloads/epi_r.csv"
# metadata = pd.read_csv(FILENAME)
# print("columns", dataset.columns.tolist())
# metadata.head(1)

In [5]:
JSON_FILE = "./drive/MyDrive/recipe_ml/full_format_recipes.json"
dataset = pd.read_json(JSON_FILE)
dataset.head()

,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
0,"[1. Place the stock, lentils, celery, carrot, ...",7.0,2006-09-01 04:00:00+00:00,"[Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...",426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap","[4 cups low-sodium vegetable or chicken stock,...",559.0
1,[Combine first 9 ingredients in heavy medium s...,23.0,2004-08-20 04:00:00+00:00,"[Food Processor, Onion, Pork, Bake, Bastille D...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"[1 1/2 cups whipping cream, 2 medium onions, c...",1439.0
2,[In a large heavy saucepan cook diced fennel a...,7.0,2004-08-20 04:00:00+00:00,"[Soup/Stew, Dairy, Potato, Vegetable, Fennel, ...",165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"[1 fennel bulb (sometimes called anise), stalk...",165.0
3,[Heat oil in heavy large skillet over medium-h...,NaN,2009-03-27 04:00:00+00:00,"[Fish, Olive, Tomato, Sauté, Low Fat, Low Cal,...",NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,"[2 tablespoons extra-virgin olive oil, 1 cup c...",NaN
4,[Preheat oven to 350°F. Lightly grease 8x8x2-i...,32.0,2004-08-20 04:00:00+00:00,"[Cheese, Dairy, Pasta, Vegetable, Side, Bake, ...",547.0,None,20.0,3.125,Spinach Noodle Casserole,"[1 12-ounce package frozen spinach soufflé, th...",452.0


In [6]:

def embed(sentence):
    return USE([sentence])

def format(data):
    ingredients = " ".join(data['ingredients']) if data['ingredients'] else ""
    steps = " ".join(data['directions']) if data['directions'] else ""
    recipe = f"TITLE: {data['title']} \nDESCRIPTION: {data['desc']} \nINGREDIENTS: {ingredients} \nINSTRUCTIONS: {steps}"
    return embed(recipe)

### clean the data

In [7]:
print("missing directions", dataset['directions'].isnull().sum())
dataset = dataset[dataset['directions'].notna()]
print("missing ingredients", dataset['ingredients'].isnull().sum())
print("missing ratings", dataset['rating'].isna().sum())
dataset = dataset[dataset['rating'].notna()]


missing directions 19
missing ingredients 0
missing ratings 11


In [73]:

clean=dataset[['rating']]
clean['recipe'] = dataset.apply(format, axis=1)
clean['rating'] /= 5


type(clean)

/tmp/ipykernel_20186/2742227155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['recipe'] = dataset.apply(format, axis=1)
/tmp/ipykernel_20186/2742227155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['rating'] /= 5


pandas.core.frame.DataFrame

In [61]:

class DataFrameDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Assuming 'recipe' is an EagerTensor
        recipe_eager_tensor = self.dataframe.iloc[idx]['recipe']
        # Convert the EagerTensor to a NumPy array, then to a PyTorch tensor
        recipe_tensor = torch.from_numpy(recipe_eager_tensor.numpy()).float()
        
        rating = self.dataframe.iloc[idx]['rating']
        # Convert rating to a PyTorch tensor, specifying the dtype as float
        rating_tensor = torch.tensor(rating, dtype=torch.float)
        
        return recipe_tensor, rating_tensor


clean_data = DataFrameDataset(clean)

In [62]:
# 70% of processed_dataset is used for training
train_size = int(0.76 * len(clean_data))
# 15% of clean_data is used for validation
val_size = int(0.12 * len(clean_data))
# The remaining 15% is used for testing
test_size = len(clean_data) - train_size - val_size


train_data, val_data, test_data = torch.utils.data.random_split(
   clean_data, [train_size, val_size, test_size]
)

print(f"Total size: {len(clean_data)}")
print(f"Train size: {len(train_data)}")
print(f"Test size: {len(test_data)}")
print(f"Val size: {len(val_data)}")


Total size: 20100
Train size: 15276
Test size: 2412
Val size: 2412


In [69]:
import torch.nn.functional as F

class StarModel(nn.Module):
    def __init__(self):
        super(StarModel, self).__init__()
        # Input layer: 512 neurons
        self.fc1 = nn.Linear(512, 512)
        # Hidden layer: 512 neurons
        self.fc2 = nn.Linear(512, 512)
        # Output layer: 1 neuron
        self.fc3 = nn.Linear(512, 1)

    def forward(self, x):
        # Pass data through fc1
        x = self.fc1(x)
        # Use ReLU activation function
        x = F.relu(x)
        # Pass data through fc2
        x = self.fc2(x)
        # UsF.ReLU activation function
        x = F.relu(x)
        # Pass data through fc3
        x = self.fc3(x)
        return x

# Create an instance of the network
net = StarModel()
print(net)

StarModel(
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1, bias=True)
)


In [82]:
print(type(train_data))

# Create a DataLoader with batch size of 32
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Define a loss function - Mean Squared Error is used here as an example
criterion = nn.MSELoss()

# Define an optimizer - Stochastic Gradient Descent is used here as an example
optimizer = optim.SGD(net.parameters(), lr=0.005)

# Number of epochs to train for
epochs = 10

model = StarModel()
model.train()


<class 'torch.utils.data.dataset.Subset'>


StarModel(
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1, bias=True)
)

In [83]:


# Train the model
for epoch in range(epochs):
    for inputs, targets in train_loader:

        # print(inputs.shape, targets.shape)
        # print("inputs", inputs)
        # print("target", targets, targets.unsqueeze(1))
        # break

        # print(inputs, targets)
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, targets.unsqueeze(1))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch+1}/{epochs} Loss: {loss.item()}")



Epoch 1/10 Loss: 0.01760173588991165
Epoch 2/10 Loss: 0.013613768853247166
Epoch 3/10 Loss: 0.01944335177540779
Epoch 4/10 Loss: 0.02120095118880272


Epoch 10/10 Loss: 0.0101045873016119
